# Лабораторная работа: Мета-обучение
## Шаг 9: Эксперимент с шумом

In [ ]:
import sys
sys.path.append('..')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from sklearn.datasets import load_iris

from src.meta_features.fixed_extractor import FixedMetaFeatureExtractor
from src.experiments.noise_generator import NoiseGenerator

%matplotlib inline

In [ ]:
# Загрузка данных
iris = load_iris()
X, y = iris.data, iris.target
extractor = FixedMetaFeatureExtractor()
noise_gen = NoiseGenerator(random_state=42)

# Оригинальные признаки
original = extractor.extract(X, y, 'original')
print("Оригинальные признаки:")
for k in ['n_instances', 'mean_mean', 'class_entropy', 'correlation_mean']:
    if k in original:
        print(f"  {k}: {original[k]:.4f}")

In [ ]:
# Добавление шума
X_noisy, y_noisy = noise_gen.add_all_noise(X, y, {
    'gaussian_level': 0.1,
    'label_noise_ratio': 0.1
})

noisy = extractor.extract(X_noisy, y_noisy, 'noisy')

print("\nИзменения после добавления шума:")
changes = {}
for k in ['n_instances', 'mean_mean', 'class_entropy', 'correlation_mean']:
    if k in original and k in noisy:
        change = noisy[k] - original[k]
        changes[k] = change
        print(f"  {k}: {original[k]:.4f} -> {noisy[k]:.4f} (изм: {change:+.4f})")

In [ ]:
# Визуализация изменений
features = []
change_values = []
for k in original:
    if k in noisy and k not in ['dataset_name']:
        if isinstance(original[k], (int, float)) and isinstance(noisy[k], (int, float)):
            features.append(k)
            change_values.append(noisy[k] - original[k])

# Берем топ-10 по абсолютному изменению
top_idx = np.argsort(np.abs(change_values))[-10:]
top_features = [features[i] for i in top_idx]
top_changes = [change_values[i] for i in top_idx]

plt.figure(figsize=(10, 6))
colors = ['red' if x < 0 else 'blue' for x in top_changes]
plt.barh(range(len(top_features)), top_changes, color=colors, alpha=0.7)
plt.yticks(range(len(top_features)), top_features)
plt.xlabel('Изменение значения')
plt.title('Топ-10 наиболее изменившихся мета-признаков')
plt.axvline(x=0, color='black', linestyle='-', linewidth=0.5)
plt.grid(True, alpha=0.3, axis='x')
plt.tight_layout()
plt.show()

### Выводы:
1. Шум влияет на структурные признаки (class_entropy, correlation_mean) сильнее всего
2. Базовые признаки (n_instances) не изменились
3. Статистические признаки (mean_mean) изменились незначительно